In [1]:
!pip install wikipedia

  Preparing metadata (setup.py) ... done
  Created wheel for wikipedia: filename=wikipedia-1.4.0-py3-none-any.whl size=11678 sha256=e27a363c21b9e08655445ac5cc87c9677fe406668fd93343249f233df34a18a1
  Stored in directory: /root/.cache/pip/wheels/5e/b6/c5/93f3dec388ae76edc830cb42901bb0232504dfc0df02fc50de
Successfully built wikipedia


In [2]:
import nltk
import numpy as np
import pandas as pd
import tensorflow as tf
import string
import math
from sklearn.preprocessing import OneHotEncoder
import matplotlib.pyplot as plt
import re
import scipy
import math
import wikipedia
import requests
from collections import Counter
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer,PorterStemmer
from nltk.corpus import stopwords
from gensim.models import Word2Vec
from sklearn.manifold import MDS
from keras.layers import Input, Dense, Dropout, Flatten, Conv2D, MaxPooling2D
from keras.models import Model
from bs4 import BeautifulSoup
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from gensim.models.fasttext import FastText

2024-05-10 18:04:27.319392: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-05-10 18:04:27.319484: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-05-10 18:04:27.432407: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


# **GET DOCS FROM WIKIPEDIA**

In [3]:
titles=wikipedia.search('spiderman')
result=wikipedia.page(titles)

In [4]:
print("len of docs",len(result.content))
print("type of docs",type(result.content))
print("content",result.content)

len of docs 66358
type of docs <class 'str'>
content The Amazing Spider-Man is a 2012 American superhero film based on the Marvel Comics character Spider-Man which shares the title of the longest-running Spider-Man comic book series. It was produced by Columbia Pictures in association with Marvel Entertainment, Laura Ziskin Productions, Arad Productions, Inc., and  Matt Tolmach Productions, and distributed by Sony Pictures Releasing. It is the fourth theatrical Spider-Man film, serving as a reboot following Sam Raimi's 2002–2007 Spider-Man trilogy, and was followed by a second The Amazing Spider-Man film. The film was directed by Marc Webb and written by James Vanderbilt, Alvin Sargent, and Steve Kloves, based on a story by Vanderbilt, and stars Andrew Garfield as Peter Parker / Spider-Man alongside Emma Stone, Rhys Ifans, Denis Leary, Campbell Scott, Irrfan Khan, Martin Sheen, and Sally Field.
Development of the film began following the cancellation of Spider-Man 4 in January 2010, en

In [5]:
def charbased_preprocessing(content):
    tokens = []
    for char in content:
        tokens.append(char)
    return(tokens)
    

In [7]:
chars=np.array(charbased_preprocessing(result.content)).reshape(-1,1)

In [8]:
encoder = OneHotEncoder()
chars = encoder.fit_transform(chars).toarray()
print(chars)

[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


In [9]:
chars.shape

(66358, 95)

In [10]:
chars[:10]

array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.

In [11]:
x=[]
y=[]
sequence_length = 10
for i in range(0,chars.shape[0]-sequence_length):
    x.append(chars[i:i+sequence_length])
    y.append(chars[i+sequence_length])
x=np.array(x)
y=np.array(y)

In [12]:
print(x.shape)
print(y.shape)

(66348, 10, 95)
(66348, 95)


In [13]:
dic = set([c for c in result.content])
dic=sorted(dic)
index2char = dict((i, c) for i, c in enumerate(dic))

In [14]:
index2char

{0: '\n',
 1: ' ',
 2: '!',
 3: '"',
 4: '#',
 5: '$',
 6: '%',
 7: '&',
 8: "'",
 9: '(',
 10: ')',
 11: '*',
 12: '+',
 13: ',',
 14: '-',
 15: '.',
 16: '/',
 17: '0',
 18: '1',
 19: '2',
 20: '3',
 21: '4',
 22: '5',
 23: '6',
 24: '7',
 25: '8',
 26: '9',
 27: ':',
 28: ';',
 29: '=',
 30: '?',
 31: '@',
 32: 'A',
 33: 'B',
 34: 'C',
 35: 'D',
 36: 'E',
 37: 'F',
 38: 'G',
 39: 'H',
 40: 'I',
 41: 'J',
 42: 'K',
 43: 'L',
 44: 'M',
 45: 'N',
 46: 'O',
 47: 'P',
 48: 'Q',
 49: 'R',
 50: 'S',
 51: 'T',
 52: 'U',
 53: 'V',
 54: 'W',
 55: 'X',
 56: 'Y',
 57: 'Z',
 58: '[',
 59: ']',
 60: 'a',
 61: 'b',
 62: 'c',
 63: 'd',
 64: 'e',
 65: 'f',
 66: 'g',
 67: 'h',
 68: 'i',
 69: 'j',
 70: 'k',
 71: 'l',
 72: 'm',
 73: 'n',
 74: 'o',
 75: 'p',
 76: 'q',
 77: 'r',
 78: 's',
 79: 't',
 80: 'u',
 81: 'v',
 82: 'w',
 83: 'x',
 84: 'y',
 85: 'z',
 86: '£',
 87: 'à',
 88: 'ç',
 89: 'è',
 90: 'é',
 91: '–',
 92: '—',
 93: '’',
 94: '−'}

In [15]:
import tensorflow as tf

# Define the size of the input and output layers
input_size = chars.shape[1] 
output_size = chars.shape[1]
# Define the hyperparameters
hidden_size = 100  # Size of the hidden state
sequence_length = 10  # Length of the input sequence
learning_rate = 0.001

# Define the RNN model with multiple layers
model = tf.keras.Sequential([
    tf.keras.layers.SimpleRNN(hidden_size, return_sequences=True, input_shape=(sequence_length, input_size)),
    tf.keras.layers.SimpleRNN(hidden_size, return_sequences=True),
    tf.keras.layers.SimpleRNN(hidden_size),
    tf.keras.layers.Dense(output_size, activation='softmax')
])

# Compile the model
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Print the model summary
model.summary()

/opt/conda/lib/python3.10/site-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ simple_rnn (SimpleRNN)          │ (None, 10, 100)        │        19,600 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ simple_rnn_1 (SimpleRNN)        │ (None, 10, 100)        │        20,100 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ simple_rnn_2 (SimpleRNN)        │ (None, 100)            │        20,100 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 95)             │         9,595 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 69,395 (271.07 KB)

 Trainable params: 69,395 (271.07 KB)

 Non-trainable params: 0 (0.00 B)

In [16]:
history=model.fit(x, y, batch_size=64, epochs=50)

Epoch 1/50
  36/1037 ━━━━━━━━━━━━━━━━━━━━ 4s 4ms/step - accuracy: 0.1019 - loss: 3.9770    

I0000 00:00:1715364303.649702     123 device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


1037/1037 ━━━━━━━━━━━━━━━━━━━━ 12s 6ms/step - accuracy: 0.2308 - loss: 2.9779
Epoch 2/50
1037/1037 ━━━━━━━━━━━━━━━━━━━━ 4s 4ms/step - accuracy: 0.3870 - loss: 2.2113
Epoch 3/50
1037/1037 ━━━━━━━━━━━━━━━━━━━━ 4s 4ms/step - accuracy: 0.4359 - loss: 2.0164
Epoch 4/50
1037/1037 ━━━━━━━━━━━━━━━━━━━━ 5s 4ms/step - accuracy: 0.4677 - loss: 1.8889
Epoch 5/50
1037/1037 ━━━━━━━━━━━━━━━━━━━━ 4s 4ms/step - accuracy: 0.4954 - loss: 1.7917
Epoch 6/50
1037/1037 ━━━━━━━━━━━━━━━━━━━━ 4s 4ms/step - accuracy: 0.5114 - loss: 1.7330
Epoch 7/50
1037/1037 ━━━━━━━━━━━━━━━━━━━━ 5s 4ms/step - accuracy: 0.5226 - loss: 1.6795
Epoch 8/50
1037/1037 ━━━━━━━━━━━━━━━━━━━━ 4s 4ms/step - accuracy: 0.5385 - loss: 1.6199
Epoch 9/50
1037/1037 ━━━━━━━━━━━━━━━━━━━━ 4s 4ms/step - accuracy: 0.5450 - loss: 1.5952
Epoch 10/50
1037/1037 ━━━━━━━━━━━━━━━━━━━━ 4s 4ms/step - accuracy: 0.5598 - loss: 1.5517
Epoch 11/50
1037/1037 ━━━━━━━━━━━━━━━━━━━━ 5s 4ms/step - accuracy: 0.5695 - loss: 1.5107
Epoch 12/50
1037/1037 ━━━━━━━━━━━━━━━━━━

# **Predict Next Char**

In [17]:
# Choose a random sequence from x
random_index = np.random.randint(0, len(x))
random_sequence = x[random_index]

# Reshape the random sequence to fit the model input shape
random_sequence = np.reshape(random_sequence, (1, sequence_length, input_size))

predicted_char_prob = model.predict(random_sequence)
predicted_char_index = np.argmax(predicted_char_prob)

# Convert the predicted character index to the actual character using index2char dictionary
predicted_char = index2char[predicted_char_index]

# Convert the random sequence back to characters using index2char dictionary
sequence_chars = [index2char[np.argmax(char)] for char in random_sequence[0]]

# Print the random sequence and the predicted character
print("Random Sequence:", ''.join(sequence_chars))
print("Predicted Next Character:", predicted_char)


1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 745ms/step
Random Sequence: m to air i
Predicted Next Character: n


# **Generate a Text**

In [18]:
sequence_index = np.random.randint(0, len(chars) - sequence_length)
sequence_chars = [index2char[np.argmax(char)] for char in chars[sequence_index:sequence_index + sequence_length]]

generated_text = ''.join(sequence_chars) 
random_index = np.random.randint(0, len(x))
random_sequence = x[random_index]

for _ in range(100):
    current_sequence = np.reshape(random_sequence, (1, sequence_length, input_size))
    predicted_char_prob = model.predict(current_sequence)
    predicted_char_index = np.argmax(predicted_char_prob)
    predicted_char = index2char[predicted_char_index]
    generated_text += predicted_char
    encoded_char=np.zeros(random_sequence.shape[1])
    encoded_char[predicted_char_index]=1
    random_sequence =np.vstack((random_sequence[1:],encoded_char))

print("Generated Text:", generated_text)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━

In [19]:
history=model.fit(x, y, batch_size=64, epochs=150)


Epoch 1/150
1037/1037 ━━━━━━━━━━━━━━━━━━━━ 5s 4ms/step - accuracy: 0.6614 - loss: 1.1309
Epoch 2/150
1037/1037 ━━━━━━━━━━━━━━━━━━━━ 4s 4ms/step - accuracy: 0.6653 - loss: 1.1259
Epoch 3/150
1037/1037 ━━━━━━━━━━━━━━━━━━━━ 5s 4ms/step - accuracy: 0.6646 - loss: 1.1281
Epoch 4/150
1037/1037 ━━━━━━━━━━━━━━━━━━━━ 5s 4ms/step - accuracy: 0.6674 - loss: 1.1121
Epoch 5/150
1037/1037 ━━━━━━━━━━━━━━━━━━━━ 5s 4ms/step - accuracy: 0.6665 - loss: 1.1081
Epoch 6/150
1037/1037 ━━━━━━━━━━━━━━━━━━━━ 4s 4ms/step - accuracy: 0.6659 - loss: 1.1148
Epoch 7/150
1037/1037 ━━━━━━━━━━━━━━━━━━━━ 4s 4ms/step - accuracy: 0.6709 - loss: 1.1040
Epoch 8/150
1037/1037 ━━━━━━━━━━━━━━━━━━━━ 5s 4ms/step - accuracy: 0.6669 - loss: 1.1108
Epoch 9/150
1037/1037 ━━━━━━━━━━━━━━━━━━━━ 4s 4ms/step - accuracy: 0.6648 - loss: 1.1070
Epoch 10/150
1037/1037 ━━━━━━━━━━━━━━━━━━━━ 4s 4ms/step - accuracy: 0.6675 - loss: 1.1113
Epoch 11/150
1037/1037 ━━━━━━━━━━━━━━━━━━━━ 4s 4ms/step - accuracy: 0.6725 - loss: 1.0949
Epoch 12/150
1037/1

In [20]:
random_sequence = x[random_index]

for _ in range(100):
    current_sequence = np.reshape(random_sequence, (1, sequence_length, input_size))
    predicted_char_prob = model.predict(current_sequence)
    predicted_char_index = np.argmax(predicted_char_prob)
    predicted_char = index2char[predicted_char_index]
    generated_text += predicted_char
    encoded_char=np.zeros(random_sequence.shape[1])
    encoded_char[predicted_char_index]=1
    random_sequence =np.vstack((random_sequence[1:],encoded_char))

print("Generated Text:", generated_text)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━